# Dataset Exploration - Consumidor.gov.br

Blablabla

## Setup

Import libraries & config

In [3]:
import pandas

pandas.set_option('display.max_columns', 50)

data_folder = '../data/consumidor-gov-br'
base_file = 'dadosgovbr---2017.csv'
sample_file = 'sample.csv'
working_file = 'dgb-2017.csv'

### Creating working file

Create Sample file

In [28]:
! head -n 1000 {data_folder}/{base_file} > {data_folder}/{sample_file}

Check charset

In [29]:
! chardetect {data_folder}/{sample_file}

../data/consumidor-gov-br/sample.csv: Windows-1252 with confidence 0.73


Convert to UTF-8 using iconv (Reference for accepted input https://gist.github.com/hakre/4188459)

In [30]:
! iconv -f CP1252 -t UTF-8 {data_folder}/{base_file} > {data_folder}/{working_file}

Check Delimiter and pattern

In [31]:
! head -n 2 {data_folder}/{working_file}

### Load Data

In [4]:
consumidor_df = pandas.read_csv(f'{data_folder}/{working_file}', delimiter=';')

## Data Quality & Analysis

In [5]:
consumidor_df.head()

,Gestor,Canal de Origem,Região,UF,Cidade,Sexo,Faixa Etária,Ano Abertura,Mês Abertura,Data Abertura,Hora Abertura,Data Resposta,Hora Resposta,Data Análise,Hora Análise,Data Recusa,Hora Recusa,Data Finalização,Hora Finalização,Prazo Resposta,Prazo Analise Gestor,Tempo Resposta,Nome Fantasia,Segmento de Mercado,Área,Assunto,Grupo Problema,Problema,Como Comprou Contratou,Procurou Empresa,Respondida,Situação,Avaliação Reclamação,Nota do Consumidor,Análise da Recusa,Edição de Conteúdo,Interação do Gestor,Total
0,Secretaria Nacional do Consumidor,Plataforma Web,N,AM,Manaus,M,entre 31 a 40 anos,2017,1,30/01/2017,14:04:07,09/02/2017,08:36:17,NaN,NaN,NaN,NaN,09/02/2017,11:43:19,09/02/2017,NaN,10.0,Bradesco Promotora,"Bancos, Financeiras e Administradoras de Cartão",Serviços Financeiros,Crédito Consignado (Empréstimo descontado em f...,Cobrança / Contestação,"Cálculo de juros, saldo devedor (contestação, ...",Loja física,S,S,Finalizada avaliada,Não Resolvida,1.0,NaN,N,N,1
1,Departamento de Proteção e Defesa do Consumido...,Plataforma Web,S,RS,Santa Cruz do Sul,F,entre 51 a 60 anos,2017,1,30/01/2017,14:25:22,08/02/2017,16:34:13,NaN,NaN,NaN,NaN,09/02/2017,11:34:58,09/02/2017,NaN,9.0,Pontofrio.com,Comércio Eletrônico,Produtos Eletrodomésticos e Eletrônicos,"Eletroportáteis ( batedeira, liquidificador, u...",Contrato / Oferta,Oferta não cumprida / serviço não fornecido/ v...,Internet,S,S,Finalizada avaliada,Não Resolvida,1.0,NaN,N,N,1
2,Instituto de Defesa do Consumidor - Procon,Plataforma Web,CO,DF,Brasília,F,entre 31 a 40 anos,2017,1,30/01/2017,20:05:01,03/02/2017,11:00:38,NaN,NaN,NaN,NaN,24/02/2017,00:02:27,09/02/2017,NaN,4.0,Samsung,"Fabricantes - Eletroeletrônicos, Produtos de ...",Produtos de Telefonia e Informática,Aparelho celular,Vício de Qualidade,Produto danificado / não funciona - Dificuldad...,Loja física,S,S,Finalizada não avaliada,Não Avaliada,NaN,NaN,N,N,1
3,Instituto Municipal de Proteção e Defesa do Co...,Plataforma Web,SE,RJ,Rio de Janeiro,F,entre 41 a 50 anos,2017,1,30/01/2017,12:41:50,09/02/2017,18:47:52,NaN,NaN,NaN,NaN,11/02/2017,15:56:05,09/02/2017,NaN,10.0,Oi Fixo,"Operadoras de Telecomunicações (Telefonia, Int...",Telecomunicações,Pacote de Serviços ( Combo ),Contrato / Oferta,Oferta não cumprida / serviço não fornecido/ v...,Não comprei / contratei,S,S,Finalizada avaliada,Resolvida,2.0,NaN,N,N,1
4,Departamento de Proteção e Defesa do Consumido...,Plataforma Web,S,RS,Porto Alegre,M,entre 41 a 50 anos,2017,1,30/01/2017,22:13:19,31/01/2017,19:22:55,NaN,NaN,NaN,NaN,21/02/2017,00:01:57,09/02/2017,NaN,1.0,Samsung,"Fabricantes - Eletroeletrônicos, Produtos de ...",Produtos de Telefonia e Informática,Aparelho celular,Contrato / Oferta,Oferta não cumprida / serviço não fornecido/ v...,Loja física,S,S,Finalizada não avaliada,Não Avaliada,NaN,NaN,N,N,1


At the first look, the information that seens interesting are:

- Gestor (which organism is following the case)
- Canal de Origem (device used to access)
- Região, UF and Cidade (Region information)
- Sexo and Faixa Etária (Demographic information)
- Ano Abertura, Mês Abertura, Data Abertura (Date information)
- Tempo de Resposta (Time to answer the complain)
- Nome Fantasia (The company receiving the complaint)
- Segmento de Mercado (The segment of the company)
- Área ()
- Assunto (Complaint reason)
- Respondida (The complaint was answered)
- Avaliação Reclamação (The evaluation from the consumer to the company answer to the complaint)
- Nota do Consumidor (The grade the consumer give to the complaint)

### Checking for null and ditribution for each of the Interesting fields

In [6]:
def check_null_and_distribution(field):    
    print(f'Check null for {field}:\n\n')

    null_total_in_field = len(consumidor_df[consumidor_df[field].isna()].index)
    print(f'TOTAL NULL: {null_total_in_field}')
    
    field_len = len(list(consumidor_df[field].unique()))
    print(f'TOTAL of unique values in field: {field_len}')
    
    dist_df = consumidor_df \
        .groupby([field]) \
        .agg({'Total': 'sum'}) \
        .sort_values(by='Total', ascending=False)
    print(dist_df.head())
    if field_len > 10:
        print(dist_df.tail())
        
    print('\n\n')

In [7]:
for field in ['Gestor', 'Canal de Origem','Região', 'Sexo', 'Faixa Etária', 'Ano Abertura', 'Mês Abertura', 
              'Data Abertura', 'Tempo Resposta', 'Nome Fantasia', 'Segmento de Mercado', 'Área', 'Assunto',
              'Respondida', 'Avaliação Reclamação', 'Nota do Consumidor']:
    check_null_and_distribution(field)

Check null for Gestor:


TOTAL NULL: 0
TOTAL of unique values in field: 21
                                                     Total
Gestor                                                    
Fundação de Proteção e Defesa do Consumidor         160115
Secretaria Nacional do Consumidor                    61896
Departamento Estadual de Proteção e Defesa do C...   49243
Programa Estadual de Proteção e Defesa do Consu...   44071
Instituto Municipal de Proteção e Defesa do Con...   27231
                                                    Total
Gestor                                                   
Programa Estadual de Proteção e Defesa do Consu...   6044
Superintendencia para Orientação e Defesa do Co...   5521
Superintendência de Proteção e Defesa do Consum...   3158
Coordenação Estadual do Procon de Rondônia           2340
Diretoria de Proteção e Defesa do Consumidor         1713



Check null for Canal de Origem:


TOTAL NULL: 0
TOTAL of unique values in field: 3
                    

**Analisys Conclusion**

The database seens to have a great quality. No inconsistences were found, the date fields have the exact number 
of unique values we expected and the other fields have no duplication, all have consistent values that make
sense.

### Creating a response_on_time column

One opportunity we identified is having a field showing if the answer was on time or not. The dataset documentation (http://dados.gov.br/dataset/reclamacoes-do-consumidor-gov-br1) say the desired time to answer is 10 days and there are values higher than that.

In [69]:
def is_reponse_on_time(time_to_answer):
    return True if time_to_answer <= 10 else False

consumidor_df['response_on_time'] = consumidor_df['Tempo Resposta'].apply(is_reponse_on_time)

In [75]:
consumidor_df.head()

,Gestor,Canal de Origem,Região,UF,Cidade,Sexo,Faixa Etária,Ano Abertura,Mês Abertura,Data Abertura,Hora Abertura,Data Resposta,Hora Resposta,Data Análise,Hora Análise,Data Recusa,Hora Recusa,Data Finalização,Hora Finalização,Prazo Resposta,Prazo Analise Gestor,Tempo Resposta,Nome Fantasia,Segmento de Mercado,Área,Assunto,Grupo Problema,Problema,Como Comprou Contratou,Procurou Empresa,Respondida,Situação,Avaliação Reclamação,Nota do Consumidor,Análise da Recusa,Edição de Conteúdo,Interação do Gestor,Total,response_on_time
0,Secretaria Nacional do Consumidor,Plataforma Web,N,AM,Manaus,M,entre 31 a 40 anos,2017,1,30/01/2017,14:04:07,09/02/2017,08:36:17,NaN,NaN,NaN,NaN,09/02/2017,11:43:19,09/02/2017,NaN,10.0,Bradesco Promotora,"Bancos, Financeiras e Administradoras de Cartão",Serviços Financeiros,Crédito Consignado (Empréstimo descontado em f...,Cobrança / Contestação,"Cálculo de juros, saldo devedor (contestação, ...",Loja física,S,S,Finalizada avaliada,Não Resolvida,1.0,NaN,N,N,1,True
1,Departamento de Proteção e Defesa do Consumido...,Plataforma Web,S,RS,Santa Cruz do Sul,F,entre 51 a 60 anos,2017,1,30/01/2017,14:25:22,08/02/2017,16:34:13,NaN,NaN,NaN,NaN,09/02/2017,11:34:58,09/02/2017,NaN,9.0,Pontofrio.com,Comércio Eletrônico,Produtos Eletrodomésticos e Eletrônicos,"Eletroportáteis ( batedeira, liquidificador, u...",Contrato / Oferta,Oferta não cumprida / serviço não fornecido/ v...,Internet,S,S,Finalizada avaliada,Não Resolvida,1.0,NaN,N,N,1,True
2,Instituto de Defesa do Consumidor - Procon,Plataforma Web,CO,DF,Brasília,F,entre 31 a 40 anos,2017,1,30/01/2017,20:05:01,03/02/2017,11:00:38,NaN,NaN,NaN,NaN,24/02/2017,00:02:27,09/02/2017,NaN,4.0,Samsung,"Fabricantes - Eletroeletrônicos, Produtos de ...",Produtos de Telefonia e Informática,Aparelho celular,Vício de Qualidade,Produto danificado / não funciona - Dificuldad...,Loja física,S,S,Finalizada não avaliada,Não Avaliada,NaN,NaN,N,N,1,True
3,Instituto Municipal de Proteção e Defesa do Co...,Plataforma Web,SE,RJ,Rio de Janeiro,F,entre 41 a 50 anos,2017,1,30/01/2017,12:41:50,09/02/2017,18:47:52,NaN,NaN,NaN,NaN,11/02/2017,15:56:05,09/02/2017,NaN,10.0,Oi Fixo,"Operadoras de Telecomunicações (Telefonia, Int...",Telecomunicações,Pacote de Serviços ( Combo ),Contrato / Oferta,Oferta não cumprida / serviço não fornecido/ v...,Não comprei / contratei,S,S,Finalizada avaliada,Resolvida,2.0,NaN,N,N,1,True
4,Departamento de Proteção e Defesa do Consumido...,Plataforma Web,S,RS,Porto Alegre,M,entre 41 a 50 anos,2017,1,30/01/2017,22:13:19,31/01/2017,19:22:55,NaN,NaN,NaN,NaN,21/02/2017,00:01:57,09/02/2017,NaN,1.0,Samsung,"Fabricantes - Eletroeletrônicos, Produtos de ...",Produtos de Telefonia e Informática,Aparelho celular,Contrato / Oferta,Oferta não cumprida / serviço não fornecido/ v...,Loja física,S,S,Finalizada não avaliada,Não Avaliada,NaN,NaN,N,N,1,True
